In [9]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.nn.modules.loss import _Loss 
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

In [10]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [11]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [12]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

In [13]:
# Initialize generator 
import ULite
from thop import profile

#input = torch.randn(1, 3, 256, 256)
#generator = ULite.ULite().cpu()
#flops, params = profile(generator, inputs=(input, ))
#print('flops:{}'.format(flops))
#print('params:{}'.format(params))


generator = ULite.ULite().cuda()
generator.load_state_dict(torch.load("./LightUNet_170.pth"))

<All keys matched successfully>

In [14]:
generator.eval()

ULite(
  (conv_in): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=same)
  (e1): EncoderBlock(
    (dw): AxialDW(
      (dw_h): Conv2d(16, 16, kernel_size=(7, 1), stride=(1, 1), padding=same, groups=16)
      (dw_w): Conv2d(16, 16, kernel_size=(1, 7), stride=(1, 1), padding=same, groups=16)
      (CAB): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): ReLU(inplace=True)
          (2): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): Sigmoid()
        )
      )
    )
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
    (down): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (act): GELU(approximate='none')
  )
  (e3): EncoderBlock(
    (dw): AxialDW(
      (dw_h): Conv2d(64, 64

In [16]:
path='C:/Users/MrYan/Desktop/underwater_program/vision/UIE/code/LSUI/Test/test/'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
i=1
print(len(path_list))
for item in path_list:
    print(item)
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx).to('cuda')
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx)
    #print(imgx.shape)
    output=generator(imgx)
    out=output.data
    save_image(out, 'C:/Users/MrYan/Desktop/underwater_program/vision/UIE/code/LSUI/Test/output/'+item, nrow=5, normalize=True)
    i=i+1


855
1.jpg
2.jpg
3.jpg
7.jpg
9.jpg
11.jpg
14.jpg
20.jpg
33.jpg
36.jpg
41.jpg
43.jpg
45.jpg
57.jpg
62.jpg
64.jpg
68.jpg
74.jpg
78.jpg
93.jpg
96.jpg
106.jpg
115.jpg
120.jpg
124.jpg
134.jpg
138.jpg
161.jpg
165.jpg
166.jpg
168.jpg
172.jpg
173.jpg
181.jpg
184.jpg
185.jpg
191.jpg
201.jpg
203.jpg
206.jpg
208.jpg
215.jpg
219.jpg
227.jpg
235.jpg
244.jpg
245.jpg
246.jpg
248.jpg
254.jpg
257.jpg
259.jpg
260.jpg
266.jpg
276.jpg
277.jpg
281.jpg
288.jpg
289.jpg
290.jpg
292.jpg
310.jpg
314.jpg
316.jpg
320.jpg
321.jpg
325.jpg
328.jpg
342.jpg
354.jpg
357.jpg
361.jpg
363.jpg
366.jpg
370.jpg
373.jpg
374.jpg
383.jpg
389.jpg
393.jpg
399.jpg
404.jpg
405.jpg
408.jpg
418.jpg
419.jpg
420.jpg
422.jpg
428.jpg
429.jpg
431.jpg
437.jpg
444.jpg
454.jpg
458.jpg
459.jpg
460.jpg
466.jpg
469.jpg
476.jpg
481.jpg
484.jpg
499.jpg
511.jpg
515.jpg
518.jpg
529.jpg
534.jpg
535.jpg
546.jpg
552.jpg
567.jpg
570.jpg
571.jpg
573.jpg
575.jpg
581.jpg
588.jpg
597.jpg
601.jpg
606.jpg
611.jpg
614.jpg
622.jpg
636.jpg
644.jpg
647.jpg
649.jp

In [12]:
import math

def compute_psnr(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
def compute_mse(img1,img2):
    mse=np.mean( (img1/255. - img2/255.) ** 2 )
    return mse

In [15]:
path1='C:/Users/MrYan/Desktop/underwater_program/vision/UIE/code/LSUI/Test/output/'#要改
path2='C:/Users/MrYan/Desktop/underwater_program/vision/UIE/code/LSUI/Test/gt/'#要改
path_list1 = os.listdir(path1)
path_list1.sort(key=lambda x:int(x.split('.')[0]))
path_list2 = os.listdir(path2)
path_list2.sort(key=lambda x:int(x.split('.')[0]))
PSNR=[]
SSIM=[]

from skimage.metrics import structural_similarity as ssim

for item in path_list1:
    impath1=path1+item
    impath2=path2+item
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    
    #print(imgx.shape)
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])

    #imgx=imgx.transpose(2,0,1)
    #imgy=imgy.transpose(2,0,1)
    
    s_value1 = ssim(imgx,imgy,channel_axis=2)
    

    psnr=(psnr1+psnr2+psnr3)/3.0
    Ssim = s_value1
    PSNR.append(psnr)
    SSIM.append(Ssim)



In [16]:
PSNR=np.array(PSNR)    
SSIM = np.array(SSIM)
print(PSNR.mean())
print(SSIM.mean())

25.458796807526905
0.8684312375823267
